In [1]:
# from feature_extraction_DEBUG import train_test_features, dumpFeatures
import _pickle as cPickle
from models import *
import spacy
import numpy as np
import pandas as pd
import csv

%load_ext autoreload
%autoreload 2


import sys
from preprocess import clean
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
import numpy as np
import _pickle as cPickle

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# train_test_features(full=False, n_gram=True, pretrained=False, nb_words=None)
# full=False
# n_gram=True
# pretrained=False
# nb_words=None

full=False
n_gram=False
pretrained=True
nb_words=None

np.random.seed(0)

if full:
    posfile = open("train_pos_full.txt", 'rb')
    negfile = open("train_neg_full.txt", 'rb')
    train_size = 2500000
else:
    posfile = open("train_pos.txt", 'rb')
    negfile = open("train_neg.txt", 'rb')
    train_size = 200000

train_tweets = []
count = 0

print('Processing of' + str(train_size) + 'tweets for the training set...')
for tweet in posfile:
    tweet = tweet.decode('utf8')
    tweet = clean(tweet)
    train_tweets.append(tweet)
    count = count + 1
    if count%1000 == 0:
        sys.stdout.write('\r{0}'.format(count) + '/' + str(train_size) + ' processed tweets.')
        sys.stdout.flush()
posfile.close()

for tweet in negfile:
    tweet = tweet.decode('utf8')
    tweet = clean(tweet)
    train_tweets.append(tweet)
    count = count + 1
    if count%1000 == 0:
        sys.stdout.write('\r{0}'.format(count) + '/' + str(train_size) + ' processed tweets.')
        sys.stdout.flush()
negfile.close()

test_size = 10000
test_tweets = []
testfile = open('test_data.txt' ,'rb')
count = 0
print('\nProcessing of' + str(test_size) + 'tweets for the testing set...')
for tweet in testfile:
    tweet = tweet.decode('utf8')
    tweet = tweet[tweet.find(',')+1:]
    tweet = clean(tweet)
    test_tweets.append(tweet)
    count = count + 1
    if count%1000 == 0:
        sys.stdout.write('\r{0}'.format(count) + '/' + str(test_size) + ' processed tweets.')
        sys.stdout.flush()
 

Processing of200000tweets for the training set...
200000/200000 processed tweets.
Processing of10000tweets for the testing set...
10000/10000 processed tweets.

In [6]:
print('\nTokenization of tweets...')
if nb_words == None:
    tokenizer = Tokenizer(filters='')
else:
    tokenizer = Tokenizer(nb_words=nb_words, filters='')
tokenizer.fit_on_texts(train_tweets)
word_index = tokenizer.word_index
max_features = len(word_index)
print('Found %s unique tokens.' % max_features)

train_sequences = tokenizer.texts_to_sequences(train_tweets)
test_sequences = tokenizer.texts_to_sequences(test_tweets)

maxlen = 30


Tokenization of tweets...
Found 96338 unique tokens.


In [7]:
len(train_sequences[0]), maxlen

(27, 30)

In [35]:
train_sequences[0]

[1,
 2,
 1562,
 508,
 450,
 15,
 999,
 88,
 45,
 7,
 133,
 508,
 14,
 240,
 855,
 78,
 26,
 6,
 37,
 2,
 148,
 5,
 72,
 63,
 17,
 371,
 680,
 684414,
 352965,
 601558,
 671564,
 580715,
 153878,
 365341,
 346731,
 301239,
 458892,
 742205,
 661487,
 389342,
 659993,
 728957,
 642261,
 677684,
 372336,
 391883,
 575777,
 120126,
 663719,
 276152,
 395968,
 281563,
 737304]

In [56]:
ngram_set
max_features, token_indice
# indice_token
# max_features

(808681,
 {(20838, 65): 96339,
  (722, 452): 96340,
  (14270, 9820): 96341,
  (278, 4317): 96342,
  (42906, 16): 96343,
  (156, 6042): 96344,
  (1347, 2340): 96345,
  (2695, 69047): 96346,
  (44, 1347): 96347,
  (25, 701): 96348,
  (2535, 96): 96349,
  (11644, 7358): 96350,
  (19884, 140): 96351,
  (1347, 3713): 96352,
  (44, 2982): 96353,
  (94, 509): 96354,
  (4230, 735): 96355,
  (3102, 1090): 96356,
  (1950, 28): 96357,
  (834, 44): 96358,
  (34, 9989): 96359,
  (1275, 12054): 96360,
  (96, 16507): 96361,
  (1735, 687): 96362,
  (4516, 17637): 96363,
  (56, 323): 96364,
  (14272, 4267): 96365,
  (1015, 141): 96366,
  (1032, 12720): 96367,
  (3, 2233): 96368,
  (2969, 17): 96369,
  (8504, 103): 96370,
  (187, 536): 96371,
  (3658, 7): 96372,
  (597, 93): 96373,
  (19218, 7): 96374,
  (16489, 29): 96375,
  (3, 4626): 96376,
  (1144, 4979): 96377,
  (433, 265): 96378,
  (31, 1348): 96379,
  (7596, 5292): 96380,
  (84, 102): 96381,
  (38703, 82): 96382,
  (430, 78): 96383,
  (80828, 23

In [4]:
if n_gram:
    maxlen = 60
    ngram_range = 2
    print('Adding {}-gram features'.format(ngram_range))
    # Create set of unique n-gram from the training set.
    ngram_set = set()
    for input_list in train_sequences:
        for i in range(2, ngram_range + 1):
            set_of_ngram = create_ngram_set(input_list, ngram_value=i)
            ngram_set.update(set_of_ngram)

    # Dictionary mapping n-gram token to a unique integer.
    # Integer values are greater than max_features in order
    # to avoid collision with existing features.
    start_index = max_features + 1
    token_indice = {v: k + start_index for k, v in enumerate(ngram_set)}
    indice_token = {token_indice[k]: k for k in token_indice} # swap keys with items

    # max_features is the highest integer that could be found in the dataset.
    max_features = np.max(list(indice_token.keys())) + 1

    # Augmenting X_train and X_test with n-grams features
    train_sequences = add_ngram(train_sequences, token_indice, ngram_range)
    test_sequences = add_ngram(test_sequences, token_indice, ngram_range)
    print('Average train sequence length: {}'.format(np.mean(list(map(len, train_sequences)), dtype=int)))
    print('Average test sequence length: {}'.format(np.mean(list(map(len, test_sequences)), dtype=int)))

In [8]:
print('Pad sequences (samples x time)')
train_sequences = sequence.pad_sequences(train_sequences, maxlen=maxlen)
test_sequences = sequence.pad_sequences(test_sequences, maxlen=maxlen)
print('train_sequences shape:', train_sequences.shape)
print('test_sequences shape:', test_sequences.shape)

Pad sequences (samples x time)
train_sequences shape: (200000, 30)
test_sequences shape: (10000, 30)


In [9]:
labels = np.array(int(train_size/2) * [0] + int(train_size/2) * [1])

print('Shuffling of the training set...')
indices = np.arange(train_sequences.shape[0])
np.random.shuffle(indices)
train_sequences = train_sequences[indices]
labels = labels[indices]

Shuffling of the training set...


In [35]:
len(train_sequences), max_features

# embeddings_index.get('<user>'.encode('utf8'), -1)
embeddings_index.get('<user>')
# line.decode('utf8')

array([ 3.1553e-01,  5.3765e-01,  1.0177e-01,  3.2553e-02,  3.7980e-03,
        1.5364e-02, -2.0344e-01,  3.3294e-01, -2.0886e-01,  1.0061e-01,
        3.0976e-01,  5.0015e-01,  3.2018e-01,  1.3537e-01,  8.7039e-03,
        1.9110e-01,  2.4668e-01, -6.0752e-02, -4.3623e-01,  1.9302e-02,
        5.9972e-01,  1.3444e-01,  1.2801e-02, -5.4052e-01,  2.7387e-01,
       -1.1820e+00, -2.7677e-01,  1.1279e-01,  4.6596e-01, -9.0685e-02,
        2.4253e-01,  1.5654e-01, -2.3618e-01,  5.7694e-01,  1.7563e-01,
       -1.9690e-02,  1.8295e-02,  3.7569e-01, -4.1984e-01,  2.2613e-01,
       -2.0438e-01, -7.6249e-02,  4.0356e-01,  6.1582e-01, -1.0064e-01,
        2.3318e-01,  2.2808e-01,  3.4576e-01, -1.4627e-01, -1.9880e-01,
        3.3232e-02, -8.4885e-01, -2.5684e-01,  2.6369e-01,  2.9562e-01,
        1.8470e-01, -2.0668e-01, -1.3297e-02,  1.2233e-01, -4.7751e-01,
       -1.7202e-01, -1.4577e-01,  4.7446e-02, -1.5824e-01,  5.4215e-02,
       -1.9426e-01, -8.1484e-02,  9.9009e-02,  1.0159e-01,  4.35

In [31]:
if pretrained:
    print('Extracting word vectors from glove200.txt...')
    embeddings_index = {}
    #f = open('embeddings/glove200.txt','rb')
    f = open('embeddings/glove.twitter.27B.200d.txt','rb')
    for line in f:
        values = line.split()
        word = values[0].decode('utf8')
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()

    print('Found %s word vectors.' % len(embeddings_index))

    print('Creating the embedding matrix...')
    embedding_matrix = np.zeros((max_features + 1, 200))
    for word, i in word_index.items():
        if i > max_features:
#             print('continue..')
            continue
        embedding_vector = embeddings_index.get(word)
#         print("embedding vector: ", embedding_vector)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    print('Embedding matrix created.')

Extracting word vectors from glove200.txt...
Found 1193514 word vectors.
Creating the embedding matrix...
Embedding matrix created.


In [36]:
for word, i in word_index.items():
    print(word, i)
    break

<user> 1


In [39]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.31553   ,  0.53764999,  0.10177   , ..., -0.45664001,
        -0.46540001,  0.52065998],
       [ 0.056404  ,  0.49535999,  0.18438999, ...,  0.63598001,
        -0.18880001, -0.035558  ],
       ...,
       [ 0.18979999,  0.14488   , -0.022718  , ...,  0.59649003,
         0.72447002, -0.15207   ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.1244    ,  0.5927    ,  0.0018391 , ...,  0.78258997,
         0.054213  , -0.56081998]])